MAKE SURE YOU HAVE NECESSARY REQUIREMENTS AND CORPUSES AS TEI FILES

In [16]:
import csv
import sys
import copy
from pathlib import Path
from itertools import combinations
from collections import defaultdict

import pandas as pd
import networkx as nx
from bs4 import BeautifulSoup

SHAKESPEAR_GENRES = {"a-midsummer-nights-dream": "Comedy",
                     "alls-well-that-ends-well": "Comedy",
                     "antony-and-cleopatra": "Tragedy",
                     "as-you-like-it": "Comedy",
                     "coriolanus": "Tragedy",
                     "cymbeline": "Tragedy",
                     "hamlet": "Tragedy",
                     "henry-iv-part-i": "Tragedy",
                     "henry-iv-part-1": "Tragedy",
                     "henry-iv-part-ii": "Tragedy",
                     "henry-iv-part-2": "Tragedy",
                     "henry-v": "Tragedy",
                     "henry-vi-part-1": "Tragedy",
                     "henry-vi-part-2": "Tragedy",
                     "henry-vi-part-3": "Tragedy",
                     "henry-viii": "Tragedy",
                     "julius-caesar": "Tragedy",
                     "king-john": "Tragedy",
                     "king-lear": "Tragedy",
                     "loves-labors-lost": "Comedy",
                     "macbeth": "Tragedy",
                     "measure-for-measure": "Comedy",
                     "much-ado-about-nothing": "Comedy",
                     "othello": "Tragedy",
                     "pericles": "Tragedy",
                     "richard-ii": "Tragedy",
                     "richard-iii": "Tragedy",
                     "romeo-and-juliet": "Tragedy",
                     "the-comedy-of-errors": "Comedy",
                     "the-merchant-of-venice": "Comedy",
                     "the-merry-wives-of-windsor": "Comedy",
                     "the-taming-of-the-shrew": "Comedy",
                     "the-tempest": "Comedy",
                     "the-winters-tale": "Comedy",
                     "timon-of-athens": "Tragedy",
                     "titus-andronicus": "Tragedy",
                     "troilus-and-cressida": "Tragedy",
                     "twelfth-night": "Comedy",
                     "two-gentlemen-of-verona": "Comedy"
                     }


**FILES**

In [2]:
shakespear_tei_files = '/home/misinagy/Projects/shakedracor/tei'
gerdracor_tei_files = 'gerdracor/tei'
original_gerdracor_metadata_file = 'gerdracor-metadata.csv'

**HELPERS**

In [12]:
def get_largest_G(input_G, name=None):
    """
    Function to extract largest connected section of nx Graph object.
    """
    if len(input_G.nodes) == 0:
        raise ValueError(f'ZERO NODE GRAPH PASSED TO get_largest_G - {input_G}: {name}')
    if nx.is_connected(input_G) is False:
        # larges connected section
        nodes_in_largest = max(nx.connected_components(input_G), key=len)
        nodes_to_remove = set(input_G.nodes) - nodes_in_largest
        G_copy = input_G.copy()
        G_copy.remove_nodes_from(nodes_to_remove)
        return G_copy
    else:
        return input_G

<h3>SHAKESPEAR ANALYSIS</h3>

In [3]:
# open xml files and parse soups
shake_soups = {}
for xml_path in Path(shakespear_tei_files).glob('*.xml'):
    print(xml_path)
    with open(xml_path, 'r') as fh:
        shake_soups[xml_path.stem] = BeautifulSoup(fh.read(), 'lxml-xml')

/home/misinagy/Projects/shakedracor/tei/a-midsummer-nights-dream.xml
/home/misinagy/Projects/shakedracor/tei/the-tempest.xml
/home/misinagy/Projects/shakedracor/tei/two-gentlemen-of-verona.xml
/home/misinagy/Projects/shakedracor/tei/the-taming-of-the-shrew.xml
/home/misinagy/Projects/shakedracor/tei/pericles.xml
/home/misinagy/Projects/shakedracor/tei/coriolanus.xml
/home/misinagy/Projects/shakedracor/tei/the-merchant-of-venice.xml
/home/misinagy/Projects/shakedracor/tei/macbeth.xml
/home/misinagy/Projects/shakedracor/tei/julius-caesar.xml
/home/misinagy/Projects/shakedracor/tei/henry-iv-part-1.xml
/home/misinagy/Projects/shakedracor/tei/the-winters-tale.xml
/home/misinagy/Projects/shakedracor/tei/henry-v.xml
/home/misinagy/Projects/shakedracor/tei/much-ado-about-nothing.xml
/home/misinagy/Projects/shakedracor/tei/henry-vi-part-1.xml
/home/misinagy/Projects/shakedracor/tei/hamlet.xml
/home/misinagy/Projects/shakedracor/tei/richard-iii.xml
/home/misinagy/Projects/shakedracor/tei/antony-

In [4]:
# get significant characters who have their own dialouge - THIS IS FOR FURTHER ANALYSIS, NOT FOR MATCHING WHOLE DRACORE NETWORK
significant_characters = []
for sp in shake_soups['coriolanus'].find_all('sp', {'who':True}):
    speakers = sp['who'].split(' ')
    if len(speakers) == 1 and speakers[0] not in significant_characters:
        significant_characters.extend(speakers)

In [4]:
# Iterate div type="act" tags and extract all characters, create combinations of 2 and add combination to list of edges
G_list = {}
for name, soup in shake_soups.items():
    edge_list = []
    lonely_nodes = []
    significant_characters = []
    for scene in soup.find_all('div', {'type': 'scene'}):
        scene_characters = []
        for sp in scene.find_all('sp', {'who': True}):
            speakers = sp['who'].split(' ')

            for split_sp in speakers:
                scene_characters.append(split_sp)
            
        if len(scene_characters) == 1:
            lonely_nodes.append(scene_characters[0])
            
        scene_edge_list = list(combinations(set(scene_characters), 2))
        edge_list += scene_edge_list
    
    whole = nx.from_edgelist(set(edge_list))
    
    for lonely_node in lonely_nodes:
        if lonely_node not in whole.nodes:
            whole.add_node(lonely_node)
    
    G_list[name] = {'whole': whole, 
                    'title_pretty': soup.find('title').get_text(strip=True), 
                    'kept_characters': ', '.join(list(whole.nodes())),
                    'character_count': len(whole.nodes()),
                   }
    

for name, soup in shake_soups.items():
    for n in [1,2,3,4,5]:
        lonely_nodes = []
        edge_list = []
    
        for act in soup.find_all('div', {'type': 'act'}):
            if act['n'] != str(n):
                for scene in act.find_all('div', {'type': 'scene'}):
                    scene_characters = []
                    for sp in scene.find_all('sp', {'who': True}):
                        speakers = sp['who'].split(' ')
                        
                        for split_sp in speakers:
                            scene_characters.append(split_sp)
                        scene_edge_list = list(combinations(set(scene_characters), 2))
                    if len(scene_characters) == 1:
                        lonely_nodes.append(scene_characters[0])
                    edge_list += scene_edge_list
        
        # create network from edge list
        whole = nx.from_edgelist(set(edge_list))
    
        # add those with independent scenes
        for lonely_node in lonely_nodes:
            if lonely_node not in whole.nodes:
                whole.add_node(lonely_node)
    
        G_list[name]['wo'+str(n)] = whole

**CSV WRITING**

In [8]:
csv_filename = 'shakedracore_metrics8.csv'
column_names = ['title', 'title_pretty', 'genre', 'kept_characters', 'character_count', 'removed_characters', 'removed_characters_count']
metric_names = ['density', 'diameter', 'average_clustering']
for w in ['whole', 'wo1', 'wo2', 'wo3', 'wo4', 'wo5']:
    for n in metric_names:
        column_names.append(w+'_'+n)

In [11]:
with open(csv_filename, 'w', newline='') as csvfile:
    fieldnames = column_names
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    
    for name, graph_dict in G_list.items():
        drama_dict = {'title': name,
                      'title_pretty': graph_dict['title_pretty'],
                      'genre': SHAKESPEAR_GENRES[name],
                      'kept_characters': graph_dict['kept_characters'],
                      'character_count': graph_dict['character_count'],
#                       'removed_characters': graph_dict['removed_characters'],
#                       'removed_characters_count': graph_dict['removed_characters_count']
                     }
        
        drama_dict['whole_density'] = nx.density(graph_dict['whole'])
        drama_dict['whole_diameter'] = nx.diameter(get_largest_G(graph_dict['whole']))
        drama_dict['whole_average_clustering'] = nx.average_clustering(graph_dict['whole'])
        
        for w in ['wo1', 'wo2', 'wo3', 'wo4', 'wo5']:
            drama_dict[w+'_density'] = nx.density(graph_dict[w])
            drama_dict[w+'_diameter'] = nx.diameter(get_largest_G(graph_dict[w]))
            drama_dict[w+'_average_clustering'] = nx.average_clustering(graph_dict[w])
        
        
        writer.writerow(drama_dict)

**CUMULATIVE CALC**

In [5]:
cumulative_G_list = defaultdict(dict)

for name, soup in shake_soups.items():
    acts_included = []
    print(f'Analysising {name} acts', end='')
    for n in ['2','3','4','5']:  # deleted '1',
        acts_included.append(n)
        print(f'{acts_included[-1]} ', end='')
        lonely_nodes = []
        edge_list = []
        significant_characters = []
    
        for act in soup.find_all('div', {'type': 'act'}):
            # div type act has an attribute 'n' with value of act number.
            if act['n'] in acts_included:
                for scene in act.find_all('div', {'type': 'scene'}):
                    scene_characters = []
                    for sp in scene.find_all('sp', {'who': True}):
                        speakers = sp['who'].split(' ')
                        if len(speakers) == 1 and speakers[0] not in significant_characters:
                            significant_characters.extend(speakers)
                        for split_sp in speakers:
                            scene_characters.append(split_sp)
                        scene_edge_list = list(combinations(set(scene_characters), 2))
                    if len(scene_characters) == 1:
                        lonely_nodes.append(scene_characters[0])
                    edge_list += scene_edge_list
        
        # create network from edge list
        n_acts_whole = nx.from_edgelist(set(edge_list))
    
        # add those with independent scenes
        for lonely_node in lonely_nodes:
            if lonely_node not in n_acts_whole.nodes:
                n_acts_whole.add_node(lonely_node)
        
        cumulative_G_list[name]['title_pretty'] = soup.find('title').get_text(strip=True)
        cumulative_G_list[name][f"acts_{'-'.join(acts_included)}"] = n_acts_whole
        

Analysising a-midsummer-nights-dream acts2 3 4 5 Analysising the-tempest acts2 3 4 5 Analysising two-gentlemen-of-verona acts2 3 4 5 Analysising the-taming-of-the-shrew acts2 3 4 5 Analysising pericles acts2 3 4 5 Analysising coriolanus acts2 3 4 5 Analysising the-merchant-of-venice acts2 3 4 5 Analysising macbeth acts2 3 4 5 Analysising julius-caesar acts2 3 4 5 Analysising henry-iv-part-1 acts2 3 4 5 Analysising the-winters-tale acts2 3 4 5 Analysising henry-v acts2 3 4 5 Analysising much-ado-about-nothing acts2 3 4 5 Analysising henry-vi-part-1 acts2 3 4 5 Analysising hamlet acts2 3 4 5 Analysising richard-iii acts2 3 4 5 Analysising antony-and-cleopatra acts2 3 4 5 Analysising richard-ii acts2 3 4 5 Analysising timon-of-athens acts2 3 4 5 Analysising troilus-and-cressida acts2 3 4 5 Analysising titus-andronicus acts2 3 4 5 Analysising romeo-and-juliet acts2 3 4 5 Analysising the-merry-wives-of-windsor acts2 3 4 5 Analysising cymbeline acts2 3 4 5 Analysising henry-vi-part-2 acts2 3

In [7]:
cumulative_G_list

defaultdict(dict,
            {'a-midsummer-nights-dream': {'title_pretty': 'A Midsummer Night’s Dream',
              'acts_2': <networkx.classes.graph.Graph at 0x786dc0d97f50>,
              'acts_2-3': <networkx.classes.graph.Graph at 0x786dc35d4da0>,
              'acts_2-3-4': <networkx.classes.graph.Graph at 0x786dec64daf0>,
              'acts_2-3-4-5': <networkx.classes.graph.Graph at 0x786de42842c0>},
             'the-tempest': {'title_pretty': 'The Tempest',
              'acts_2': <networkx.classes.graph.Graph at 0x786da1973140>,
              'acts_2-3': <networkx.classes.graph.Graph at 0x786dec10d4c0>,
              'acts_2-3-4': <networkx.classes.graph.Graph at 0x786da0c2f9e0>,
              'acts_2-3-4-5': <networkx.classes.graph.Graph at 0x786dec0db800>},
             'two-gentlemen-of-verona': {'title_pretty': 'Two Gentlemen of Verona',
              'acts_2': <networkx.classes.graph.Graph at 0x786dc352d4c0>,
              'acts_2-3': <networkx.classes.graph.Graph at 

**CSV WRITING CUMULATIVE**

In [ ]:
csv_filename = 'shakedracor_cumulative2.csv'
column_names = ['title', 'title_pretty', 'genre']
metric_names = ['density', 'diameter', 'average_clustering']
for w in ['acts_1', 'acts_1-2', 'acts_1-2-3', 'acts_1-2-3-4', 'acts_1-2-3-4-5']:
    for n in metric_names:
        column_names.append(w+'_'+n)

In [8]:
# just from 2
csv_filename = 'shakedracor_cumulative2.csv'
column_names = ['title', 'title_pretty', 'genre']
metric_names = ['density', 'diameter', 'average_clustering']
for w in ['acts_2', 'acts_2-3', 'acts_2-3-4', 'acts_2-3-4-5']:
    for n in metric_names:
        column_names.append(w+'_'+n)

In [36]:
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=column_names)
    writer.writeheader()
    
    for name, drama_data in cumulative_G_list.items():
        drama_dict = {'title': name,
              'title_pretty': drama_data['title_pretty'],
              'genre': SHAKESPEAR_GENRES[name],
             }
        for w in ['acts_1', 'acts_1-2', 'acts_1-2-3', 'acts_1-2-3-4', 'acts_1-2-3-4-5']:
            drama_dict[f'{w}_density'] = nx.density(drama_data[w])
            drama_dict[f'{w}_diameter'] = nx.diameter(get_largest_G(drama_data[w]))
            drama_dict[f'{w}_average_clustering'] = nx.average_clustering(drama_data[w])
        
        writer.writerow(drama_dict)

In [17]:
# just for from 2
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=column_names)
    writer.writeheader()
    
    for name, drama_data in cumulative_G_list.items():
        drama_dict = {'title': name,
              'title_pretty': drama_data['title_pretty'],
              'genre': SHAKESPEAR_GENRES[name],
             }
        for w in ['acts_2', 'acts_2-3', 'acts_2-3-4', 'acts_2-3-4-5']:
            drama_dict[f'{w}_density'] = nx.density(drama_data[w])
            drama_dict[f'{w}_diameter'] = nx.diameter(get_largest_G(drama_data[w]))
            drama_dict[f'{w}_average_clustering'] = nx.average_clustering(drama_data[w])
        
        writer.writerow(drama_dict)

**RESULTS**

In [ ]:
# results
for name, G in G_list.items():
    print(f'{name:>15} DENSITY:', nx.density(G))
    print(f'{name:>15} DIAMETER:', nx.diameter(G_list_connected[name]))
    print(f'{name:>15} AVERAGE CLUSTERING:', nx.average_clustering(G), '\n')

<h3>GERMAN DRAMA ANALYSIS</h3>

**GERDRACORE CUMULATIVE**

In [4]:
# open xml files and parse soups
ger_soups = {}
for xml_path in Path(gerdracor_tei_files).glob('*.xml'):
    with open(xml_path, 'r') as fh:
        # Filter 5 act, 5+ actor, Tragedy and Comedy dramas from GerDracor
        soup = BeautifulSoup(fh.read(), 'lxml-xml')
        
        acts = soup.find_all('div', {'type': 'act'})
        cast_list = soup.find('profileDesc').find('listPerson').find_all('person')
        genre_tag = soup.find('textClass').find('keywords').find('term', {'type': 'genreTitle'})
        genre = genre_tag.get_text(strip=True)
        if cast_list is None:
            raise ValueError('No cast list found!')
        if len(acts) == 5 and len(cast_list) > 5 and genre in ['Tragedy', 'Comedy']:
            ger_soups[xml_path.stem] = soup

In [65]:
s = ger_soups['sudermann-der-bettler-von-syrakus']
divs = s.body.find_all('div', recursive=False)
print(len(divs))
# len(divs[1].find_all('div', recursive=False))
# divs[1].find_all('div', recursive=False)[2]
len(s.find_all('div', {'type': 'act'}))

6


5

In [33]:
# sudermann-der-bettler-von-syrakus
# hebbel-der-diamant
# schiller-die-jungfrau-von-orleans
# hauptmann-florian-geyer
# laube-monaldeschi

for name, soup in ger_soups.items():
    
    all_divs = len(soup.body.find_all('div', recursive=False))
    
    all_acts = len(soup.find_all('div', {'type': 'act'}))
    prologue = soup.find_all('div', {'type': 'prologue'})
    epilogue = soup.find_all('div', {'type': 'epilogue'})
    
    print(name, (50-len(name))*' ', all_divs, all_acts)

grillparzer-des-meeres-und-der-liebe-wellen         5 5
goethe-clavigo                                      5 5
grillparzer-medea                                   5 5
hauptmann-carl-tobias-buntschuh                     5 5
ludwig-die-makkabaeer                               5 5
toerring-agnes-bernauerin                           5 5
beer-struensee                                      5 5
hofmannsthal-der-turm                               5 5
wieland-lady-johanna-gray                           5 5
gottschedin-das-testament                           5 5
laube-gottsched-und-gellert                         5 5
immermann-andreas-hofer                             5 5
gryphius-catharina-von-georgien                     5 5
collin-coriolan                                     5 5
goethe-der-grosskophta                              5 5
hebbel-judith                                       5 5
grabbe-kaiser-heinrich-der-sechste                  5 5
klinger-das-leidende-weib                       

In [54]:
def one_appearance_unit_edge_list(unit, lonely_nodes):
    """
    Take unit of BS4 tag and extract all connections between speakers in unit.
    """
    characters = []
                
    for sp in unit.find_all('sp', {'who': True}):
        # get each speaker from speaker tag
        speakers = sp['who'].split(' ')

        for split_sp in speakers:
            characters.append(split_sp)
    edge_list = list(combinations(set(characters), 2))
    # if only one character in scene, add lonely node
    if len(characters) == 1:
        lonely_nodes.append(characters[0])
    
    return edge_list, lonely_nodes

In [52]:
# CREATE THE MAIN EXTRACTOR FUNCTION
# take first n acts from drama
def edge_list_extractor_extractor(list_of_soup_segments):
    """
    Takes list of soup elements, returns edge list for shared scenes, and if no scenes, just shared acts.
    This is GerDracor specific, and should only be used for the 128 dramas --> len(acts) == 5 and len(cast_list) > 5 and genre in ['Tragedy', 'Comedy']
    """
    
    lonely_nodes = []
    edge_list_in_iteration = []

    # iterate over n acts
    for c, act in enumerate(list_of_soup_segments, start=1):
        if act is None:
            if c == 1 or c == len(list_of_soup_segments):
                continue
            else:
                raise ValueError(f'ACT {c} IS NONE IN {name}')
                
        configurations_in_act = act.find_all('div', {'type': 'configuration'})
        locations_in_act = act.find_all('div', {'type': 'location'})
        scenes_in_act = act.find_all('div', {'type': 'scene'})
        
        check_list = [len(configurations_in_act) > 0, len(locations_in_act) > 0, len(scenes_in_act) > 0]
        if len([c for c in check_list if c is True]) > 1:
            raise ValueError(f'UNACCOUNTED STRUCTURE OF DRAMA - {name} ! {check_list}')
        
        # IF IT HAS SCENES
        if len(scenes_in_act) > 0:
            for scene in scenes_in_act:
                if scene.find('div', {'type': 'scene'}) is not None:
                    for inner_scene in scene.find_all('div', {'type': 'scene'}):
                        if inner_scene.find('div', {'type': 'scene'}):
                            print(f'SCENE WITHIN SCENE WITHIN SCENE in {name}', file=sys.stderr)
                        scene_edge_list, lonely_nodes = one_appearance_unit_edge_list(inner_scene, lonely_nodes)
                        edge_list_in_iteration += scene_edge_list
                else:
                    scene_edge_list, lonely_nodes = one_appearance_unit_edge_list(scene, lonely_nodes)
                    edge_list_in_iteration += scene_edge_list

        # IT HAS LOCATIONS
        elif len(locations_in_act) > 0:
            for location in locations_in_act:
                    
                location_edge_list, lonely_nodes = one_appearance_unit_edge_list(location, lonely_nodes)
                edge_list_in_iteration += location_edge_list
        
        # IT HAS CONFIGURATIONS
        elif len(configurations_in_act) > 0:
            for configuration in configurations_in_act:
                configuration_edge_list, lonely_nodes = one_appearance_unit_edge_list(configuration, lonely_nodes)
                edge_list_in_iteration += configuration_edge_list
        
        # IT HAS NO SCENES AND NO LOCATIONS AND NO CONFIGURATIONS
        else:
            if act.find('div') is not None:
                raise ValueError(f'Unaccounted div type in {name} !')
            act_edge_list, lonely_nodes = one_appearance_unit_edge_list(act, lonely_nodes)
            edge_list_in_iteration += act_edge_list

    return edge_list_in_iteration, lonely_nodes

In [82]:
# TEST
# sudermann-der-bettler-von-syrakus
# hebbel-der-diamant
# schiller-die-jungfrau-von-orleans
# hauptmann-florian-geyer
# laube-monaldeschi

In [125]:
# MAIN CUMULATIVE NETWORKS GENERATION FOR GERDRACOR

cumulative_G_list_ger = defaultdict(dict)

for name, soup in ger_soups.items():
    
    # Metadata annotation for dict
    cumulative_G_list_ger[name]['soup'] = soup
    genre_tag = soup.find('textClass').find('keywords').find('term', {'type': 'genreTitle'})
    cumulative_G_list_ger[name]['genre'] = genre_tag.get_text(strip=True)
    cumulative_G_list_ger[name]['title_pretty'] = soup.find('title').get_text(strip=True)
    
    # These are used to determine what structural elements there are.
    all_acts = soup.find_all('div', {'type': 'act'})
    prologue = soup.find('div', {'type': 'prologue'})
    epilogue = soup.find('div', {'type': 'epilogue'})
    
    # Whole calculation to 
    whole_list = []
    whole_list.extend(all_acts)
    
    if prologue is not None:
        whole_list.insert(0, prologue)
    elif prologue is None:
        whole_list.insert(0, None)
    
    if epilogue is not None:
        whole_list.append(epilogue)
    elif epilogue is None:
        whole_list.append(None)
        
    edge_list, lonely_nodes = edge_list_extractor_extractor(whole_list)
    # create network from edge list
    whole_drama = nx.from_edgelist(set(edge_list))

    # add those with independent scenes
    for lonely_node in lonely_nodes:
        if lonely_node not in whole_drama.nodes:
            whole_drama.add_node(lonely_node)
    
    cumulative_G_list_ger[name]['whole'] = whole_drama
    
    # Culminative calculation
    print(f'Processing {name}', (50-(len(name)))*' ', end='')
    # create iterations for 1, 1-2, 1-2-3, ... acts
    for iteration_round in range(1, len(whole_list)+1):  # was all_acts
        
        # take first n acts from drama
        acts_included = whole_list[:iteration_round]  # was all_acts
        print(f'-{len(acts_included)}', end='')
        
        edge_list_in_iteration, lonely_nodes = edge_list_extractor_extractor(acts_included)

        # create network from edge list
        n_acts_whole = nx.from_edgelist(set(edge_list_in_iteration))

        # add those with independent scenes
        for lonely_node in lonely_nodes:
            if lonely_node not in n_acts_whole.nodes:
                n_acts_whole.add_node(lonely_node)
        
        if iteration_round == 1:
            label = 'epilogue'
            if whole_list[0] is None:
                n_acts_whole = None
        elif iteration_round == len(whole_list):
            label = 'prologue'
            if whole_list[-1] is None:
                n_acts_whole = None
        else:
            label = f"acts_{'-'.join([str(i) for i in range(1, iteration_round)])}"
            
        cumulative_G_list_ger[name][label] = n_acts_whole
    print('\n---------------')

Processing grillparzer-des-meeres-und-der-liebe-wellen        -1-2-3-4-5-6-7
---------------
Processing goethe-clavigo                                     -1-2-3-4-5-6-7
---------------
Processing grillparzer-medea                                  -1-2-3-4-5-6-7
---------------
Processing hauptmann-carl-tobias-buntschuh                    -1-2-3-4-5-6-7
---------------
Processing ludwig-die-makkabaeer                              -1-2-3-4-5-6-7
---------------
Processing toerring-agnes-bernauerin                          -1-2-3-4-5-6-7
---------------
Processing beer-struensee                                     -1-2-3-4-5-6-7
---------------
Processing hofmannsthal-der-turm                              -1-2-3-4-5-6-7
---------------
Processing wieland-lady-johanna-gray                          -1-2-3-4-5-6-7
---------------
Processing gottschedin-das-testament                          -1-2-3-4-5-6-7
---------------
Processing laube-gottsched-und-gellert                        -1-2-3-4

In [120]:
# MAIN CUMULATIVE NETWORKS GENERATION FOR GERDRACOR WITHOUT EPILOGUE AND PROLOGUE

cumulative_G_list_ger = defaultdict(dict)

for name, soup in ger_soups.items():

    # Metadata annotation for dict
    cumulative_G_list_ger[name]['soup'] = soup
    genre_tag = soup.find('textClass').find('keywords').find('term', {'type': 'genreTitle'})
    cumulative_G_list_ger[name]['genre'] = genre_tag.get_text(strip=True)
    cumulative_G_list_ger[name]['title_pretty'] = soup.find('title').get_text(strip=True)
    
    # These are used to determine what structural elements there are.
    all_acts = soup.find_all('div', {'type': 'act'})
        
    edge_list, lonely_nodes = edge_list_extractor_extractor(all_acts)
    # create network from edge list
    whole_drama = nx.from_edgelist(set(edge_list))

    # add those with independent scenes
    for lonely_node in lonely_nodes:
        if lonely_node not in whole_drama.nodes:
            whole_drama.add_node(lonely_node)
    
    cumulative_G_list_ger[name]['whole'] = whole_drama
    
    # Culminative calculation
    print(f'Processing {name}', (50-(len(name)))*' ', end='\n')
    # create iterations for 1, 1-2, 1-2-3, ... acts
    for iteration_round in range(1, len(all_acts)+1):
        
        # take first n acts from drama
        acts_included = all_acts[:iteration_round] 
        # print(f'-{len(acts_included)}', end='')
        
        edge_list_in_iteration, lonely_nodes = edge_list_extractor_extractor(acts_included)

        # create network from edge list
        print(name, len(set(edge_list_in_iteration)))
        n_acts_whole = nx.from_edgelist(set(edge_list_in_iteration))

        # add those with independent scenes
        for lonely_node in lonely_nodes:
            if lonely_node not in n_acts_whole.nodes:
                n_acts_whole.add_node(lonely_node)
        
        label = f"acts_{'-'.join([str(i) for i in range(1, iteration_round+1)])}"    
        
        cumulative_G_list_ger[name][label] = n_acts_whole
    print('\n---------------')

Processing grillparzer-des-meeres-und-der-liebe-wellen        
grillparzer-des-meeres-und-der-liebe-wellen 55
grillparzer-des-meeres-und-der-liebe-wellen 55
grillparzer-des-meeres-und-der-liebe-wellen 55
grillparzer-des-meeres-und-der-liebe-wellen 55
grillparzer-des-meeres-und-der-liebe-wellen 55

---------------
Processing goethe-clavigo                                     
goethe-clavigo 21
goethe-clavigo 28
goethe-clavigo 28
goethe-clavigo 28
goethe-clavigo 38

---------------
Processing grillparzer-medea                                  
grillparzer-medea 28
grillparzer-medea 32
grillparzer-medea 32
grillparzer-medea 43
grillparzer-medea 47

---------------
Processing hauptmann-carl-tobias-buntschuh                    
hauptmann-carl-tobias-buntschuh 6
hauptmann-carl-tobias-buntschuh 18
hauptmann-carl-tobias-buntschuh 59
hauptmann-carl-tobias-buntschuh 64
hauptmann-carl-tobias-buntschuh 66

---------------
Processing ludwig-die-makkabaeer                              
ludwig-die-ma

In [121]:
cumulative_G_list_ger['grillparzer-des-meeres-und-der-liebe-wellen']['acts_1-2'].edges
ger_soups['grillparzer-des-meeres-und-der-liebe-wellen'].body.find_all('div', {'type': 'act'})[0]

<div type="act">
<head>Erster Aufzug</head>
<stage>Vorhof im Tempel der Aphrodite zu Sestos. Den Mittelgrund bilden Säulen mit weiten
          Zwischenräumen, das Peristyl bezeichnend. Im Hintergrunde der Tempel, zu dem mehrere
          Stufen emporführen. Nach vorne, rechts die Statue Amors, links Hymenäus Bildsäule. Früher
          Morgen.</stage>
<sp who="#hero">
<speaker>HERO</speaker>
<stage>ein Körbchen mit Blumen im Arme haltend, tritt aus dem Tempel und steigt die Stufen
            herab.</stage>
<lg>
<l>Nun, so weit wärs getan. Geschmückt der Tempel,</l>
<l>Mit Myrt und Rosen ist er rings bestreut</l>
<l>Und harret auf das kommende, das Fest.</l>
</lg>
<lg>
<l>Und ich bin dieses Festes Gegenstand.</l>
<l>Mir wird vergönnt, die unbemerkten Tage,</l>
<l>Die fernhin rollen ohne Richt und Ziel,</l>
<l>Dem Dienst der hohen Himmlischen zu weihn;</l>
<l>Die einzelnen, die Wiesenblümchen gleich,</l>
<l>Der Fuß des Wanderers zertritt und knickt,</l>
<l>Zum Kranz gewunden um der Göt

In [124]:
# Write csv for no epilogue no prologue section

csv_filename = 'gerdracor_cumulative2_no_pro_epi.csv'
column_names = ['title', 'title_pretty', 'genre']
metric_names = ['density', 'diameter', 'average_clustering']
most_acts = max([len(v) for v in cumulative_G_list_ger.values()]) - 1

for act in range(1, 5+1):
    for n in metric_names:
        column_names.append(f"acts_{'-'.join([str(n) for n in range(1, act+1)])}_{n}")

ger_result_dict = defaultdict(dict)
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=column_names)
    writer.writeheader()
    
    for name, drama_data in cumulative_G_list_ger.items():
        drama_dict = {'title': name,
                      'title_pretty': drama_data['title_pretty'],
                      'genre': drama_data['genre']
             }
        
        
        for act in range(1, 5+1):
            acts_name = f"acts_{'-'.join([str(n) for n in range(1, act+1)])}"
            if acts_name in drama_data.keys():
                drama_dict[f"{acts_name}_density"] = nx.density(drama_data[acts_name])
                drama_dict[f'{acts_name}_diameter'] = nx.diameter(get_largest_G(drama_data[acts_name], name))
                drama_dict[f'{acts_name}_average_clustering'] = nx.average_clustering(drama_data[acts_name])
            else:
                raise ValueError(f'{acts_name} not in {name}')
        ger_result_dict[name] = drama_dict
        writer.writerow(drama_dict)

**GERDRACOR STRUCTURES**

In [150]:
def div_structure(unit, indent):
    for div in unit.find_all('div', recursive=False):
        print(f"""{indent*" "}{div['type']}""")
        div_structure(div, indent+2)
for name, soup in ger_soups.items():
    if name in ['hebbel-genoveva', 'neuber-das-schaeferfest']:
        body = soup.find('body')
        div_structure(body, 2)

  act
    scene
    scene
    scene
  act
    scene
    scene
    scene
    scene
    scene
  act
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
  act
    scene
    scene
    scene
    scene
    scene
    scene
  act
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
  epilogue
    Dramatis_Personae
    set
    scene
    scene
    scene
    scene
    scene
  act
    scene
    scene
    scene
    scene
    scene
  act
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
  act
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
  act
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
    scene
  act
    scene

In [ ]:
ger_result_dict = defaultdict(dict)
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=column_names)
    writer.writeheader()
    
    for name, drama_data in cumulative_G_list_ger.items():
        drama_dict = {'title': name,
                      'title_pretty': drama_data['title_pretty'],
                      'genre': drama_data['genre']
             }
        
        for section in ['epilogue', 'prologue']:
            if drama_data[section] is not None:
                density = nx.density(drama_data[section])
                diameter = nx.diameter(get_largest_G(drama_data[section], name))
                average_clustering = nx.average_clustering(drama_data[section])
            else:
                density, diameter, average_clustering = None, None, None
                
            drama_dict[f"{section}_density"] = density
            drama_dict[f'{section}_diameter'] = diameter
            drama_dict[f'{section}_average_clustering'] = average_clustering
        
        for act in range(1, 5+1):
            acts_name = f"acts_{'-'.join([str(n) for n in range(1, act+1)])}"
            if acts_name in drama_data.keys():
                drama_dict[f"{acts_name}_density"] = nx.density(drama_data[acts_name])
                drama_dict[f'{acts_name}_diameter'] = nx.diameter(get_largest_G(drama_data[acts_name], name))
                drama_dict[f'{acts_name}_average_clustering'] = nx.average_clustering(drama_data[acts_name])
        ger_result_dict[name] = drama_dict
        writer.writerow(drama_dict)

**GERDRACOR CSV WRITING**

In [128]:
csv_filename = 'gerdracor_cumulative3.csv'
column_names = ['title', 'title_pretty', 'genre']
metric_names = ['density', 'diameter', 'average_clustering']
most_acts = max([len(v) for v in cumulative_G_list_ger.values()]) - 1

for act in range(1, 5+1):
    for n in metric_names:
        column_names.append(f"acts_{'-'.join([str(n) for n in range(1, act+1)])}_{n}")
        
for section in ['epilogue', 'prologue']:
    for n in metric_names:
        column_names.append(f"{section}_{n}")

In [129]:
ger_result_dict = defaultdict(dict)
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=column_names)
    writer.writeheader()
    
    for name, drama_data in cumulative_G_list_ger.items():
        drama_dict = {'title': name,
                      'title_pretty': drama_data['title_pretty'],
                      'genre': drama_data['genre']
             }
        
        for section in ['epilogue', 'prologue']:
            if drama_data[section] is not None:
                density = nx.density(drama_data[section])
                diameter = nx.diameter(get_largest_G(drama_data[section], name))
                average_clustering = nx.average_clustering(drama_data[section])
            else:
                density, diameter, average_clustering = None, None, None
                
            drama_dict[f"{section}_density"] = density
            drama_dict[f'{section}_diameter'] = diameter
            drama_dict[f'{section}_average_clustering'] = average_clustering
        
        for act in range(1, 5+1):
            acts_name = f"acts_{'-'.join([str(n) for n in range(1, act+1)])}"
            if acts_name in drama_data.keys():
                drama_dict[f"{acts_name}_density"] = nx.density(drama_data[acts_name])
                drama_dict[f'{acts_name}_diameter'] = nx.diameter(get_largest_G(drama_data[acts_name], name))
                drama_dict[f'{acts_name}_average_clustering'] = nx.average_clustering(drama_data[acts_name])
        ger_result_dict[name] = drama_dict
        writer.writerow(drama_dict)

**CROSSCHECK GERDRACOR**

In [187]:
metadata_df = pd.read_csv(original_gerdracor_metadata_file)

NameError: name 'original_gerdracor_metadata_file' is not defined

In [185]:
metadata_df = metadata_df.set_index('name')

In [186]:
not_matching = set()
df_data = []
for name, drama_dict in cumulative_G_list_ger.items():
    row = {'name': name}
    
    G = drama_dict['whole']
    my_density = nx.density(G)
    my_diameter = nx.diameter(get_largest_G(G, name))
    my_average_clustering = nx.average_clustering(G)
    
    if round(my_density, 2) != round(metadata_df.loc[name]['density'], 2):
        row['density'] = metadata_df.loc[name]['density']
        row['my_density'] = my_density
    if round(my_diameter, 2) != round(metadata_df.loc[name]['diameter'], 2):
        row['diameter'] = metadata_df.loc[name]['diameter']
        row['my_diameter'] = my_diameter
    if round(my_average_clustering, 2) != round(metadata_df.loc[name]['averageClustering'], 2):
        row['avg_clu'] = metadata_df.loc[name]['averageClustering']
        row['my_avg_clu'] = my_average_clustering
    if len(row) > 1:
        df_data.append(row)
pd.DataFrame(df_data)
    

""


**GERDRACOR STATS**

In [121]:
ger_dracor_stats = []
for name, soup in ger_soups.items():
    
    acts = len(soup.find_all('div', {'type': 'act'})) > 0
    prologue = len(soup.find_all('div', {'type': 'prologue'})) > 0
    scenes = len(soup.find_all('div', {'type': 'scene'})) > 0
    configurations = len(soup.find_all('div', {'type': 'configuration'})) > 0
    locations = len(soup.find_all('div', {'type': 'location'})) > 0
    
    if all(elem is False for elem in [acts, scenes, configurations, locations]):
        raise ValueError(f'CONTENT HOLDER DIVS NOT ACCOUNTED FOR {name}')

    # has acts and scenes
    if acts and scenes:
        if scenes is False:
            raise ValueError(f'{name}')
        scene_lens_in_acts = []
        for act in soup.find_all('div', {'type': 'act'}):
            scenes_list = act.find_all('div', {'type': 'scene'})
            scene_lens_in_acts.append(len(scenes_list))
        if all(elem > 0 for elem in scene_lens_in_acts):
            all_acts_have_scenes = True
            
        elif any(elem > 0 for elem in scene_lens_in_acts):
            all_acts_have_scenes = False
    else:
        all_acts_have_scenes = False
        
    ger_dracor_stats.append({'name': name, 
                             'prologue': prologue,
                             'acts': acts, 
                             'scenes': scenes, 
                             'all_acts_have_scenes': all_acts_have_scenes,
                             'configurations': configurations, 
                             'locations': locations})
    

In [122]:
with open('gerdracor_content_stats.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['name', 'prologue', 'acts', 'scenes', 'all_acts_have_scenes', 'configurations', 'locations'])
    writer.writeheader()
    for d in ger_dracor_stats:
        writer.writerow(d)